In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [11]:
# Load and clean data
# dataset/Cleaned_Census_Dataset.csv
df = pd.read_csv('dataset/Cleaned_Census_Dataset.csv')  # Placeholder filename
df['NOC'] = df['NOC'].astype(str)
df['NOC_length'] = df['NOC'].apply(len)

# Approximate population data (hardcoded for simplicity)
population_data = {
    'Newfoundland and Labrador': 525000,
    'Prince Edward Island': 175000,
    'Nova Scotia': 1000000,
    'New Brunswick': 800000,
    'Quebec': 8700000,
    'Ontario': 14900000,
    'Manitoba': 1400000,
    'Saskatchewan': 1200000,
    'Alberta': 4600000,
    'British Columbia': 5400000,
    'Yukon': 43000,
    'Northwest Territories': 45000,
    'Nunavut': 40000
}

population_df = pd.DataFrame(list(population_data.items()), columns=['Province', 'Population'])
df = df.merge(population_df, on='Province')
df['Employment_per_10k'] = (df['Employment'] / df['Population']) * 10000
df.head()



,Province,Occupation,NOC,Gender,Employment,NOC_length,Population,Employment_per_10k
0,Newfoundland and Labrador,Occupation - not applicable 13,nan,Men,74715.0,3,525000,1423.142857
1,Newfoundland and Labrador,All occupations 14,nan,Men,136465.0,3,525000,2599.333333
2,Newfoundland and Labrador,0 Legislative and senior management occupations,0.0,Men,1270.0,3,525000,24.190476
3,Newfoundland and Labrador,00 Legislative and senior managers,0.0,Men,1270.0,3,525000,24.190476
4,Newfoundland and Labrador,000 Legislative and senior managers,0.0,Men,1270.0,3,525000,24.190476


In [19]:
# Dash app setup
app = dash.Dash(__name__)
app.title = "2023 Canadian Census Dashboard"

# App layout
app.layout = html.Div([
    html.H1("2023 Canadian Census - Employment Dashboard"),

    html.Div([
        html.Label("Select Essential Service"),
        dcc.Dropdown(
            id='essential-service-dropdown',
            options=[
                {'label': 'Registered Nurses', 'value': 'Registered nurses and registered psychiatric nurses'},
                {'label': 'Commissioned Police Officers', 'value': 'Commissioned police officers and related occupations in public protection services'},
                {'label': 'Fire Chiefs and Senior Officers', 'value': 'Fire chiefs and senior firefighting officers'},
            ],
            value='Registered nurses and registered psychiatric nurses'
        ),
    ]),

    dcc.Graph(id='essential-service-graph'),

    html.Div([
        html.Label("Select Gender for NOC Analysis"),
        dcc.RadioItems(
            id='gender-radio',
            options=[
                {'label': 'Men', 'value': 'Men'},
                {'label': 'Women', 'value': 'Women'}
            ],
            value='Men',
            inline=True
        )
    ]),

    dcc.Graph(id='gender-noc-graph'),

    html.Div([
        html.Label("Select Engineering Discipline for EV Analysis"),
        dcc.Dropdown(
            id='engineer-discipline-dropdown',
            options=[
                {'label': 'Computer Engineers', 'value': '21311'},
                {'label': 'Mechanical Engineers', 'value': '21301'},
                {'label': 'Electrical Engineers', 'value': '21310'},
            ],
            value='21311'
        )
    ]),

    dcc.Graph(id='ev-engineer-graph'),

    html.Div([
        html.Label("Select a Gender to Explore Open Insight"),
        dcc.Dropdown(
            id='open-gender-dropdown',
            options=[
                {'label': 'Men', 'value': 'Men'},
                {'label': 'Women', 'value': 'Women'}
            ],
            value='Women'
        )
    ]),

    dcc.Graph(id='open-ended-graph')
])

# Callbacks
@app.callback(
    Output('essential-service-graph', 'figure'),
    Input('essential-service-dropdown', 'value')
)
def update_essential_service_chart(selected_occupation):
    dff = df[df['Occupation'].str.contains(selected_occupation, case=False, na=False)]
    dff_grouped = dff.groupby(['Province'], as_index=False)['Employment_per_10k'].sum()
    fig = px.bar(
        dff_grouped,
        x='Province',
        y='Employment_per_10k',
        title=f"Essential Service Availability - {selected_occupation} (per 10,000)",
        labels={'Employment_per_10k': 'Employment per 10,000'}
    )
    return fig

@app.callback(
    Output('gender-noc-graph', 'figure'),
    Input('gender-radio', 'value')
)
def update_gender_noc_chart(selected_gender):
    dff = df[(df['Gender'] == selected_gender) & (df['NOC'].str[0].isin(['1', '2']))]
    dff_grouped = dff.groupby(['Province', 'Occupation'], as_index=False)['Employment_per_10k'].sum()
    fig = px.treemap(
        dff_grouped,
        path=['Province', 'Occupation'],
        values='Employment_per_10k',
        title=f"Employment Distribution by Province and Occupation (NOC starting with 1 or 2) for {selected_gender}"
    )
    return fig


@app.callback(
    Output('ev-engineer-graph', 'figure'),
    Input('engineer-discipline-dropdown', 'value')
)
def update_ev_chart(engineer_noc):
    dff = df[df['NOC'] == engineer_noc]
    fig = px.line(
        dff,
        x='Province',
        y='Employment_per_10k',
        color='Gender',
        markers=True,
        title=f"Engineer Availability (NOC {engineer_noc}) per Province",
        labels={'Employment_per_10k': 'Employment per 10,000'}
    )
    return fig

@app.callback(
    Output('open-ended-graph', 'figure'),
    Input('open-gender-dropdown', 'value')
)
def update_open_insight_chart(selected_gender):
    dff = df[(df['Gender'] == selected_gender) & (df['NOC_length'] == 1)]
    dff_grouped = dff.groupby('Province', as_index=False)['Employment_per_10k'].sum()

    if dff_grouped.empty:
        fig = px.line(title="No data available for the selected gender.")
    else:
        fig = px.pie(
            dff_grouped,
            names='Province',
            values='Employment_per_10k',
            title=f"Proportional Employment per Province for {selected_gender} (NOC 1-digit)"
        )
    return fig

# Run server
if __name__ == '__main__':
    app.run(debug=True)